In [1]:
import pathlib
import shutil

import pandas as pd

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

bandicoot_path = pathlib.Path("/home/lippincm/mnt/bandicoot").resolve()
if bandicoot_path.exists():
    # comment out depending on whose computer you are on
    # mike's computer
    bandicoot_path = pathlib.Path("/home/lippincm/mnt/bandicoot").resolve()
    # Jenna's computer
    # bandicoot_path = pathlib.Path("/media/18tbdrive/GFF_organoid_data/")
    raw_image_dir = pathlib.Path(f"{bandicoot_path}/NF1_organoid_data/").resolve()
    output_base_dir = bandicoot_path
else:
    raw_image_dir = pathlib.Path(f"{root_dir}/NF1_organoid_data/").resolve()
    output_base_dir = root_dir

In [2]:
raw_image_dir

PosixPath('/home/lippincm/mnt/bandicoot/NF1_organoid_data')

In [3]:
# get all image_paths
# deconvolution paths
data_path = pathlib.Path(
    "/home/lippincm/Desktop/20TB_A/NF1_processed_organoids/data/decon_inputs/"
).resolve(strict=True)
# get all patient dirs present
channel_dirs = [pathlib.Path(f"{p}") for p in data_path.iterdir() if p.is_dir()]
well_fov_image_paths = [
    f
    for dir in channel_dirs
    for f in dir.iterdir()
    if f.is_file() and f.suffix == ".tiff"
]
well_fov_image_paths.sort()
df = pd.DataFrame({"image_path": well_fov_image_paths})
df["file_name"] = df["image_path"].apply(lambda x: x.name)
# df['file_parent'] = df['image_path'].apply(lambda x: x.parent)
df["channel"] = df["file_name"].apply(lambda x: str(x.split("_")[3].split(".tiff")[0]))
df["patient_id"] = df["file_name"].apply(
    lambda x: str(x.split("_")[0]) + "_" + str(x.split("_")[1])
)
df["well_id"] = df["file_name"].apply(lambda x: str(x.split("_")[2]))
df["new_file_name"] = df["well_id"] + "_" + df["channel"] + ".tif"
df.head()

,image_path,file_name,channel,patient_id,well_id,new_file_name
0,/home/lippincm/Desktop/20TB_A/NF1_processed_or...,NF0014_T1_C10-1_405_cmle.ome.tiff,405,NF0014_T1,C10-1,C10-1_405.tif
1,/home/lippincm/Desktop/20TB_A/NF1_processed_or...,NF0014_T1_C10-2_405_cmle.ome.tiff,405,NF0014_T1,C10-2,C10-2_405.tif
2,/home/lippincm/Desktop/20TB_A/NF1_processed_or...,NF0014_T1_C11-1_405_cmle.ome.tiff,405,NF0014_T1,C11-1,C11-1_405.tif
3,/home/lippincm/Desktop/20TB_A/NF1_processed_or...,NF0014_T1_C11-2_405_cmle.ome.tiff,405,NF0014_T1,C11-2,C11-2_405.tif
4,/home/lippincm/Desktop/20TB_A/NF1_processed_or...,NF0014_T1_C2-1_405_cmle.ome.tiff,405,NF0014_T1,C2-1,C2-1_405.tif


In [4]:
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Copying files"):
    patient = row["patient_id"]
    well = row["well_id"]
    channel = row["channel"]
    image_path = row["image_path"]
    new_image_path = pathlib.Path(
        f"{raw_image_dir}/data/{patient}/deconvolved_images/{well}/{row['new_file_name']}"
    )
    new_image_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copyfile(image_path, new_image_path)

Copying files:   0%|          | 0/8977 [00:00<?, ?it/s]